In [1]:
import zipfile
import pandas as pd

In [2]:
# read the dataset using the compression zip
df = pd.read_csv('vehicles.zip', delimiter=',',compression='zip')

In [3]:
# display dataset
print(df.shape)

(150000, 26)
